In [ ]:
import os
import sys
from pathlib import Path
project_dir = Path().resolve().parent.parent
sys.path.append(str(project_dir))


os.environ['USE_PYGEOS'] = os.environ.get('USE_PYGEOS', '0')

import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from IPython.display import Markdown

from masterplan_tools.method import MasterPlan
from masterplan_tools.method.provision import ProvisionModel
from masterplan_tools.models import CityModel
from masterplan_tools.preprocessing import DataGetter



def pandas_to_markdown(df_or_series: pd.DataFrame | pd.Series, value_name: str | None = None) -> Markdown:
    if isinstance(df_or_series, pd.DataFrame):
        return Markdown(
            "\n".join(
                (
                    f"| {' | '.join(column for column in df_or_series.columns)} |",
                    f"| {' | '.join(('---',) * df_or_series.shape[1])} |",
                    "\n".join(
                        f"| {' | '.join(str(value) for value in values)} |" for _, values in df_or_series.iterrows()
                    ),
                )
            )
        )
    elif isinstance(df_or_series, pd.Series):
        if value_name is None:
            value_name = "value"
        return Markdown(
            "\n".join(
                (
                    f"| {df_or_series.name} | {value_name} |",
                    "| --- | --- |",
                    "\n".join(f"| {key} | {round(value)} |" for key, value in df_or_series.items()),
                )
            )
        )
    raise ValueError(f"'{df_or_series}' is neither DataFrame nor Series")

In [ ]:
# path to data
example_data_path = "./data"
# TODO: upload example data somewhere and download it in script


# load data required for blocks creation
city_geometry = gpd.read_parquet(os.path.join(example_data_path, "city_geometry.parquet"))
water_geometry = gpd.read_parquet(os.path.join(example_data_path, "water_geometry.parquet"))
roads_geometry = gpd.read_parquet(os.path.join(example_data_path, "roads_geometry.parquet"))
railways_geometry = gpd.read_parquet(os.path.join(example_data_path, "railways_geometry.parquet"))
nature_geometry_boundaries = gpd.read_parquet(os.path.join(example_data_path, "nature_geometry_boundaries.parquet"))

# load data required for service graphs creation
schools = gpd.read_parquet(os.path.join(example_data_path, "schools.parquet"))
kindergartens = gpd.read_parquet(os.path.join(example_data_path, "kindergartens.parquet"))
recreational_areas = gpd.read_parquet(os.path.join(example_data_path, "recreational_areas.parquet"))

hospitals = gpd.read_file(os.path.join(example_data_path, "hospitals.geojson"))
pharmacies = gpd.read_file(os.path.join(example_data_path, "pharmacies.geojson"))
policlinics = gpd.read_file(os.path.join(example_data_path, "policlinics.geojson"))

# load data required for
buildings = gpd.read_parquet(os.path.join(example_data_path, "buildings.parquet"))
greenings = gpd.read_parquet(os.path.join(example_data_path, "greenings.parquet"))
parkings = gpd.read_parquet(os.path.join(example_data_path, "parkings.parquet"))


transport_graph = nx.read_graphml(os.path.join(example_data_path, "new_graph.graphml"))

# data loading with planning area
polygon = gpd.read_file(os.path.join(example_data_path, "polygon.geojson"))

In [ ]:
# services should be specified as a dictionary
services = {"schools": schools, "kindergartens": kindergartens, "recreational_areas": recreational_areas,
            "hospitals": hospitals, "pharmacies": pharmacies, "policlinics": policlinics}

In [ ]:
local_crs = 32636

no_development = gpd.read_file(os.path.join(example_data_path, "no_development_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
no_development = no_development[no_development['RAYON']=='Василеостровский']
landuse = gpd.read_file(os.path.join(example_data_path, "landuse_zone_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
okn = gpd.read_file(os.path.join(example_data_path, "petropavlovsraya_delete.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)

In [ ]:
from masterplan_tools.method import BlocksCutter
from masterplan_tools.method.blocks import BlocksCutterGeometries

geoms = BlocksCutterGeometries(
  city=city_geometry,
  water=water_geometry,
  roads=roads_geometry,
  railways=railways_geometry,
  nature=nature_geometry_boundaries, 
  no_dev=no_development,
  landuse=landuse
)
blocks = BlocksCutter(geometries = geoms).get_blocks() 

In [ ]:
from masterplan_tools.method import LuFilter
"""
There are three landuse tags in the blocks gdf:
    1. 'no_dev_area' -- according to th no_debelopment gdf and cutoff without any buildings or specified / selected landuse types;
    2. 'selected_area' -- according to the landuse gdf. We separate theese polygons since they have specified landuse types;
    3. 'buildings' -- there are polygons that have buildings landuse type. 

    In further calculations we will use the in the following steps:
    Only 'buildings' -- to find clusters of buildings in big polygons;
    All of them while calculating the accessibility times among city blocks;
    All of them except 'no_dev_area' while optimizing the development of new facilities.
"""
blocks = LuFilter(blocks.to_gdf(), landuse_geometries = geoms).filter_lu()

In [ ]:
from masterplan_tools.method.blocks.blocks_clustering import BlocksClusterization

buildings_geom = gpd.read_file(os.path.join(example_data_path, "buildings_blocks.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
blocks = BlocksClusterization(blocks, buildings_geom).run()

In [ ]:

block_t = blocks
block_t.reset_index(drop=True, inplace=True)
block_t['id'] = block_t.index
block_t['index'] = block_t['id']
block_t.drop('landuse', axis=1, inplace=True)
block_t.reset_index(drop=True, inplace=True)


In [ ]:
# City data model creation
city_model = CityModel(
    services=services,
    city_blocks = block_t,
    city_geometry=city_geometry,
    water_geometry=water_geometry,
    roads_geometry=roads_geometry,
    railways_geometry=railways_geometry,
    nature_geometry_boundaries=nature_geometry_boundaries,
    # accessibility_matrix=accessibility_matrix,
    transport_graph=transport_graph,
    buildings=buildings,
    greenings=greenings,
    parkings=parkings,

)

In [ ]:
services_prov = {}

for service_type in services.keys():
    provision = ProvisionModel(city_model=city_model, service_name=service_type)
    print(service_type)
    services_prov[service_type] = provision.run()

In [ ]:
from matplotlib.gridspec import GridSpec

kindergartens_prov = services_prov["kindergartens"]
schools_prov = services_prov["schools"]
recreational_areas_prov = services_prov["recreational_areas"]
hospitals_prov = services_prov["hospitals"]
pharmacies_prov = services_prov["pharmacies"]
policlinics_prov = services_prov["policlinics"]

fig = plt.figure(figsize=(25, 15))
gs = GridSpec(2, 3, figure=fig)

ax1 = fig.add_subplot(gs[0, 0])
kindergartens_prov.plot(column="provision_kindergartens", legend=True, ax=ax1)
ax1.set_title("Kindergartens provision")
kindergartens_prov[kindergartens_prov["population"] == 0].plot(ax=ax1, color="grey", alpha=1)

ax2 = fig.add_subplot(gs[0, 1])
schools_prov.plot(column="provision_schools", legend=True, ax=ax2)
ax2.set_title("Schools provision")
schools_prov[schools_prov["population"] == 0].plot(ax=ax2, color="grey", alpha=1)

ax3 = fig.add_subplot(gs[0, 2])
recreational_areas_prov.plot(column="provision_recreational_areas", legend=True, ax=ax3)
ax3.set_title("Recreational areas provision")
recreational_areas_prov[recreational_areas_prov["population"] == 0].plot(ax=ax3, color="grey", alpha=1)

ax4 = fig.add_subplot(gs[1, 0])
hospitals_prov.plot(column="provision_hospitals", legend=True, ax=ax4)
ax4.set_title("Hospitals provision")
hospitals_prov[hospitals_prov["population"] == 0].plot(ax=ax4, color="grey", alpha=1)

ax5 = fig.add_subplot(gs[1, 1])
pharmacies_prov.plot(column="provision_pharmacies", legend=True, ax=ax5)
ax5.set_title("Pharmacies provision")
pharmacies_prov[pharmacies_prov["population"] == 0].plot(ax=ax5, color="grey", alpha=1)

ax6 = fig.add_subplot(gs[1, 2])
policlinics_prov.plot(column="provision_policlinics", legend=True, ax=ax6)
ax6.set_title("Policlinics provision")
policlinics_prov[policlinics_prov["population"] == 0].plot(ax=ax6, color="grey", alpha=1)

plt.show()

In [ ]:
prov = pd.concat([df if i==0 else df.drop(['id', 'geometry', 'population'], axis=1) for i, df in enumerate(services_prov.values())], axis=1)

In [ ]:
prov.drop('level_0', axis=1, inplace=True)
prov

In [ ]:
df = city_model.blocks_aggregated_info.merge(prov)
del prov
df

In [ ]:
df = df[['block_id', 'geometry' , 'current_population', 'population', 'area',
    'floors', 'current_living_area', 'current_industrial_area',
    'current_green_area',
    'current_green_capacity', 'current_parking_capacity',
    'population_prov_schools', 'population_unprov_schools',
    'population_prov_kindergartens', 'population_unprov_kindergartens',
    'population_prov_recreational_areas', 'population_unprov_recreational_areas',
    'population_prov_hospitals', 'population_unprov_hospitals',
    'population_prov_policlinics', 'population_unprov_policlinics',
    'population_prov_pharmacies', 'population_unprov_pharmacies']]

In [ ]:
# df.isna().sum()[df.isna().sum()!=0]
# df[df['floors'].isna()][['population', 'current_living_area']].describe()
df['floors'].fillna(0, inplace=True)
# (df['current_population'] != df['population']).sum()
df.drop(['current_population'], axis=1, inplace=True)

In [ ]:
HECTARE_IN_SQUARE_METERS = 10_000

df[['area', 'current_living_area', 'current_industrial_area', 'current_green_area']] = df[['area', 'current_living_area', 'current_industrial_area', 'current_green_area']]/HECTARE_IN_SQUARE_METERS

In [ ]:
df['free_area'] = df['area']*0.8 - df['current_green_area'] - df['current_industrial_area'] - df['current_living_area']

In [ ]:
services_dict = {
    'schools': {250: 1.2, 300: 1.1, 600: 1.3, 800: 1.5, 1100: 1.8},
    'kindergartens': {180: 0.72, 250: 1.44, 280: 1.1},
    'recreational_areas': {1000: 0.12},
    'pharmacies': {1000: 0.005},
    'hospitals': {500: 4, 1000: 8, 5000: 40, 10000: 80},
    'policlinics': {1000: 1.1, 3000: 3, 5000: 5}
}

In [ ]:
d = {}
for k,v in services_dict.items():
    for kk,vv in v.items():
        d[k+'_'+str(kk)]=vv
d_ = pd.DataFrame([d]).T

In [ ]:
d_

In [ ]:
import itertools

In [ ]:
comb = [list(itertools.combinations(list(d.keys()), i)) for i in range(1, 4)]
comb = [item for sublist in comb for item in sublist]

In [ ]:
comb_w = []
for comb_ in comb:
    comb_w.append(d_.loc[list(comb_)].sum()[0])

In [ ]:
df['variants'] = df['free_area'].apply(lambda x: [i for i,com in enumerate(comb_w) if com<=x])

In [ ]:
df['variants']

In [ ]:
total_vars = list(set([item for sublist in df['variants'].tolist() for item in sublist]))

In [ ]:
variants = pd.DataFrame(columns=services.keys(), index=range(833)).fillna(0)
for i,_ in enumerate(comb):
    for el in [x.rsplit('_', maxsplit=1) for x in _]:
        variants.loc[i, el[0]] += int(el[1])

In [ ]:
variants

In [ ]:
import numpy as np

In [ ]:
blocks = np.zeros(140)

In [ ]:
blocks_for_change = np.random.randint(0, 140, 10)
change_variants = np.random.randint(0, 832, 10)

In [ ]:
blocks[blocks_for_change] = change_variants

In [ ]:
def fitness_func(ga_instance, blocks, solution_idx):
    updated_block_dict = {}

    for i, block_id in enumerate(blocks.nonzero()[0]):
        updated_block = {
            'block_id': df.loc[block_id]['block_id'],
            'population': df.loc[block_id]['population']}
        for k,v in variants.loc[blocks[block_id]].to_dict().items():
            updated_block[k+'_capacity'] = v

        updated_block_dict[i] = updated_block

    services_graph_new = nx.Graph()
    for service_type in services.keys():
        services_graph_new = DataGetter().prepare_graph(
            blocks=city_model.city_blocks,
            service_type=service_type,
            buildings=city_model.buildings,
            service_gdf=city_model.services_gdfs[service_type],
            updated_block_info=updated_block_dict,
            accessibility_matrix=city_model.accessibility_matrix,
            services_graph=services_graph_new,
        )

    city_model.services_graph = services_graph_new

    services_prov = {}
    for service_type in services.keys():
        provision = ProvisionModel(city_model=city_model, service_name=service_type)
        temp = provision.run()
        temp = temp[temp['population']!=0]
        services_prov[service_type] = 1/6*(temp['population_prov_' + service_type]/
                                           (temp['population_prov_' + service_type] +
                                            temp['population_unprov_' + service_type])).mean()

    fitness = sum(services_prov.values())

    return fitness

In [ ]:
fitness_func(None, blocks, None)

In [ ]:
df['len_variants'] = df['variants'].apply(lambda x: len(x))

In [ ]:
import pygad

In [ ]:
ga_instance = pygad.GA(num_generations=3,
                       num_parents_mating=6,

                       fitness_func=fitness_func,

                       sol_per_pop=6,
                       num_genes=df[df['len_variants']!=0].reset_index(drop=True).shape[0],
                       gene_space=df[df['len_variants']!=0].reset_index(drop=True)['variants'].tolist(),
                       gene_type=int,

                       mutation_type='adaptive',
                       mutation_percent_genes=(90, 10),

                       # on_crossover = on_crossover,
                       crossover_type='scattered',

                       parent_selection_type='tournament',
                       K_tournament=3,

                       stop_criteria='saturate_50',
                       parallel_processing=12)

In [ ]:
# ga_instance.plot_fitness()

In [ ]:
solution = ga_instance.best_solution()[0]

In [ ]:
updated_block_dict = {}

for i, block_id in enumerate(solution.nonzero()[0]):
    updated_block = {
        'block_id': df.loc[block_id]['block_id'],
        'population': df.loc[block_id]['population']}
    for k,v in variants.loc[solution[block_id]].to_dict().items():
        updated_block[k+'_capacity'] = v

    updated_block_dict[i] = updated_block

In [ ]:
# city_model.services_graph = services_graph
# services_prov = {}
# for service_type in services.keys():
#     provision = ProvisionModel(city_model=city_model, service_name=service_type)
#     services_prov[service_type] = provision.run()

In [ ]:
comb[9]

In [ ]:
solution

In [ ]:
services_graph_new = nx.Graph()
for service_type in services.keys():
    services_graph_new = DataGetter().prepare_graph(
        blocks=city_model.city_blocks,
        service_type=service_type,
        buildings=city_model.buildings,
        service_gdf=city_model.services_gdfs[service_type],
        updated_block_info=updated_block_dict,
        accessibility_matrix=city_model.accessibility_matrix,
        services_graph=services_graph_new,
    )

city_model.services_graph = services_graph_new

services_prov_new = {}
for service_type in services.keys():
    provision = ProvisionModel(city_model=city_model, service_name=service_type)
    services_prov_new[service_type] = provision.run()

In [ ]:
from matplotlib.gridspec import GridSpec

kindergartens_prov = services_prov["kindergartens"]
schools_prov = services_prov["schools"]
recreational_areas_prov = services_prov["recreational_areas"]
hospitals_prov = services_prov["hospitals"]
pharmacies_prov = services_prov["pharmacies"]
policlinics_prov = services_prov["policlinics"]

fig = plt.figure(figsize=(25, 15))
gs = GridSpec(2, 3, figure=fig)

ax1 = fig.add_subplot(gs[0, 0])
kindergartens_prov.plot(column="provision_kindergartens", legend=True, ax=ax1)
ax1.set_title("Kindergartens provision")
kindergartens_prov[kindergartens_prov["population"] == 0].plot(ax=ax1, color="grey", alpha=1)

ax2 = fig.add_subplot(gs[0, 1])
schools_prov.plot(column="provision_schools", legend=True, ax=ax2)
ax2.set_title("Schools provision")
schools_prov[schools_prov["population"] == 0].plot(ax=ax2, color="grey", alpha=1)

ax3 = fig.add_subplot(gs[0, 2])
recreational_areas_prov.plot(column="provision_recreational_areas", legend=True, ax=ax3)
ax3.set_title("Recreational areas provision")
recreational_areas_prov[recreational_areas_prov["population"] == 0].plot(ax=ax3, color="grey", alpha=1)

ax4 = fig.add_subplot(gs[1, 0])
hospitals_prov.plot(column="provision_hospitals", legend=True, ax=ax4)
ax4.set_title("Hospitals provision")
hospitals_prov[hospitals_prov["population"] == 0].plot(ax=ax4, color="grey", alpha=1)

ax5 = fig.add_subplot(gs[1, 1])
pharmacies_prov.plot(column="provision_pharmacies", legend=True, ax=ax5)
ax5.set_title("Pharmacies provision")
pharmacies_prov[pharmacies_prov["population"] == 0].plot(ax=ax5, color="grey", alpha=1)

ax6 = fig.add_subplot(gs[1, 2])
policlinics_prov.plot(column="provision_policlinics", legend=True, ax=ax6)
ax6.set_title("Policlinics provision")
policlinics_prov[policlinics_prov["population"] == 0].plot(ax=ax6, color="grey", alpha=1)

plt.show()
